In [ ]:
!wget https://www.dropbox.com/s/dleabirlq4o3klo/Dataset.zip?dl=0  -O Dataset.zip
!unzip -q Dataset.zip

--2022-05-21 10:30:02--  https://www.dropbox.com/s/dleabirlq4o3klo/Dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dleabirlq4o3klo/Dataset.zip [following]
--2022-05-21 10:30:02--  https://www.dropbox.com/s/raw/dleabirlq4o3klo/Dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc734fb803905fb3bacb59c47387.dl.dropboxusercontent.com/cd/0/inline/BlpZruYK1dG48xZr5SMq0JtbfMhKQyjNXOCW60YNHFPf-MuqrjebbPRZWj63ZlC1tcHvWqRtY-niYz0D_6d9-qXyJLElBiso-gR3U3mFM5JgUpAtjVAOqi1uhVbFu7BfKrx8YXbiAOeeUNnu2oFDTeztXu_wzgDE11xENEOFEZ5vnA/file# [following]
--2022-05-21 10:30:03--  https://uc734fb803905fb3bacb59c47387.dl.dropboxusercontent.com/cd/0/inline/BlpZruYK1dG48xZr5SMq0JtbfMhKQyjNXOCW60YNHFPf-MuqrjebbPRZWj63ZlC

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 32.3 MB/s 
     |████████████████████████████████| 6.6 MB 63.2 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 33.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## load_corpus

In [ ]:
from collections import Counter

import pandas as pd
import torch

path = 'Dataset'
def load_corpus(path):
    """
    :param path: 样本语料库的文件
    :return: 文本内容contents，以及分类标签labels(onehot形式)
    """
    contents, labels = [], []
    with open(path, encoding='utf-8') as f:
        for line in f.readlines():
            sp = line.strip().split()
            if(len(sp) == 0):
                continue
            label = int(sp[0])
            content =''.join(i for i in sp[1:])
            labels.append(label)
            contents.append(content)
    counter = Counter(labels)
    print('总样本数为：%d' % (len(labels)))
    print('各个类别样本数如下：')
    for w in counter:
        print(w, counter[w])
    return contents, labels

## data_gen

In [ ]:
from transformers import BertTokenizer
print('下载 BERT tokenizer...')
BERT_MAX_LEN=128
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)
class Datapro:
    def __init__(self,sents):
        self.sentences=sents

    def datagen(self):
        input_ids = []
        attention_masks = []
        for sent in self.sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = tokenizer(
                sent,  # Sentence to encode.
                add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
                max_length=128,  # Pad & truncate all sentences.
                pad_to_max_length=True,
                return_attention_mask=True,  # Construct attn. masks.
                return_tensors='pt',  # Return pytorch tensors.
            )

            # 把编码的句子加入list.
            input_ids.append(encoded_dict['input_ids'])

            # 加上 attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
        return input_ids,attention_masks


下载 BERT tokenizer...


## 主函数

In [ ]:
if __name__=='__main__':
    #训练数据
    tcontents, tlabels = load_corpus(path + '/' + 'train.txt')
    tinput_ids, tattention_masks=Datapro(tcontents).datagen()
    #验证数据
    vcontents, vlabels = load_corpus(path + '/' + 'validation.txt')
    vinput_ids, vattention_masks = Datapro(vcontents).datagen()
tinput_ids = torch.cat(tinput_ids, dim=0)
tattention_masks = torch.cat(tattention_masks, dim=0)
tlabels = torch.tensor(tlabels)
print('原句: ', tcontents[0])
print('Token IDs:', tinput_ids[0])
print('attention_masks:', tattention_masks[0])
#验证集
vinput_ids = torch.cat(vinput_ids, dim=0)
vattention_masks = torch.cat(vattention_masks, dim=0)
vlabels = torch.tensor(vlabels)
print('原句: ', vcontents[0])
print('Token IDs:', vinput_ids[0])
print('attention_masks:', vattention_masks[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


总样本数为：19998
各个类别样本数如下：
1 9999
0 9999


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


总样本数为：5629
各个类别样本数如下：
1 2812
0 2817
原句:  死囚爱刽子手女贼爱衙役我们爱你们难道还有别的选择没想到胡军除了蓝宇还有东宫西宫我个去阿兰这样真他nia恶心爱个P分明只是欲
Token IDs: tensor([  101,  3647,  1723,  4263,  1175,  2094,  2797,  1957,  6592,  4263,
         6126,  2514,  2769,   812,  4263,   872,   812,  7410,  6887,  6820,
         3300,  1166,  4638,  6848,  2885,  3766,  2682,  1168,  5529,  1092,
         7370,   749,  5905,  2126,  6820,  3300,   691,  2151,  6205,  2151,
         2769,   702,  1343,  7350,  1065,  6821,  3416,  4696,   800, 10560,
         8139,  2626,  2552,  4263,   702,   158,  1146,  3209,  1372,  3221,
         3617,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,

## dataset

In [ ]:
from torch.utils.data import TensorDataset, random_split

# 把input 放入 TensorDataset。
train_dataset = TensorDataset(tinput_ids, tattention_masks, tlabels)
val_dataset = TensorDataset(vinput_ids, vattention_masks, vlabels)
# 计算 train_size 和 val_size 的长度.
train_size = int( len(train_dataset))
val_size = int( len(val_dataset))

print('{:>5,} 训练数据'.format(train_size))
print('{:>5,} 验证数据'.format(val_size))


19,998 训练数据
5,629 验证数据


## dataloader

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 推荐batch_size 为 16 或者 32
batch_size = 32

# 为训练数据集和验证数据集设计DataLoaders.
train_dataloader = DataLoader(
            train_dataset,  # 训练数据.
            sampler = RandomSampler(train_dataset), # 打乱顺序
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset, # 验证数据.
            sampler = RandomSampler(val_dataset), # 打乱顺序
            batch_size = batch_size
        )
from transformers import BertModel, BertConfig
from torch.optim import AdamW

In [ ]:
def same_seeds(seed):
    # torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # np.random.seed(seed)
    # random.seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
same_seeds(0)
model_name='bert-base-chinese'

## model

In [ ]:
import torch
from torch import nn
from transformers import BertModel
model_name = 'bert-base-chinese'

In [ ]:
class Bert_lstm(nn.Module):

    def __init__(self):
        super(Bert_lstm, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.lstm = nn.LSTM(input_size=768,hidden_size=768,batch_first=True)
        self.dense1 = nn.Linear(768, 512)
        self.dense2 = nn.Linear(512,128)
        self.dense3 = nn.Linear(128,2)

    def forward(self, input_ids,attention_mask):
        output_bert = self.bert(input_ids,attention_mask)
        output_lhs = output_bert.last_hidden_state
        output_lstm, (output_hn, output_cn) = self.lstm(output_lhs)
        output_dense1 = self.dense1(output_hn)
        output_dense2 = self.dense2(output_dense1)
        output = self.dense3(output_dense2)
        return torch.squeeze(output)

In [ ]:

from transformers import get_linear_schedule_with_warmup
from torch.nn import functional as F
# bert 推荐 epochs 在2到4之间为好。
epochs = 5

device='cuda' if torch.cuda.is_available() else 'cpu'
cri = nn.CrossEntropyLoss()
mymodel = Bert_lstm()
mymodel.to(device)
optimizer = AdamW(mymodel.parameters(),
                  lr = 1e-5 # args.learning_rate - 默认是 5e-5
               
                )


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## trainer

In [ ]:
from tqdm import tqdm
def rtrain(rcnet):
    i = 1
    model_path = 'model'
    print('执行次数为：{}'.format(i))
    rcnet = rcnet.to(device)
    rcnet.train()
    for epoch in range(epochs):
        train_loss = 0
        r_acc = 0
        step = 0
        for batch_idx, data in enumerate(tqdm(train_dataloader)):
            torch.autograd.set_detect_anomaly(True)

            inputs_id, att_mask,rel_batch = data
            inputs_id, att_mask=inputs_id.to(device), att_mask.to(device)
            optimizer.zero_grad()

            # rc model识别关系  -----------------------------------------------------
            rel_tar = rel_batch.to(device)

            rel = rcnet(inputs_id, att_mask)  # 想得到的是B*len(label2id)

            rel_loss=cri(rel,rel_tar)
            t_loss =   rel_loss
            t_loss.backward()

            train_loss += t_loss.item()

            optimizer.step()

        print(train_loss/len(train_dataset))

rtrain(mymodel)

执行次数为：1


100%|██████████| 625/625 [08:46<00:00,  1.19it/s]


0.011382690639701626


100%|██████████| 625/625 [08:51<00:00,  1.18it/s]


0.007500835387471176


...

0.00498590882009182


100%|██████████| 625/625 [08:51<00:00,  1.18it/s]


0.003143255519509764


100%|██████████| 625/625 [08:51<00:00,  1.18it/s]

0.0020078781649831906


## test

### 主函数

In [ ]:
if __name__=='__main__':
    # 测试数据
    scontents, slabels = load_corpus('Dataset' + '/' + 'test.txt')
    sinput_ids, sattention_masks=Datapro(scontents).datagen()

sinput_ids = torch.cat(sinput_ids, dim=0)
sattention_masks = torch.cat(sattention_masks, dim=0)
slabels = torch.tensor(slabels)
print('测试集 原句: ', scontents[0])
print('Token IDs:', sinput_ids[0])
print('attention_masks:', sattention_masks[0])
test_dataset = TensorDataset(sinput_ids, sattention_masks, slabels)
test_size = int( len(test_dataset))

print('{:>5,} 测试数据'.format(test_size))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


总样本数为：369
各个类别样本数如下：
1 187
0 182
测试集 原句:  如果我无聊时网上乱逛偶尔看到这部电影我可能会给它打四星但是TNND姐是花大洋去电影院看姐在电影院睡着姐现在非常心疼电影票钱一部无聊至极电影
Token IDs: tensor([ 101, 1963, 3362, 2769, 3187, 5464, 3198, 5381,  677,  744, 6859,  981,
        2209, 4692, 1168, 6821, 6956, 4510, 2512, 2769, 1377, 5543,  833, 5314,
        2124, 2802, 1724, 3215,  852, 3221,  162, 9502, 8168, 1995, 3221, 5709,
        1920, 3817, 1343, 4510, 2512, 7368, 4692, 1995, 1762, 4510, 2512, 7368,
        4717, 4708, 1995, 4385, 1762, 7478, 2382, 2552, 4563, 4510, 2512, 4873,
        7178,  671, 6956, 3187, 5464, 5635, 3353, 4510, 2512,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
attention_masks: ten

In [ ]:

batch_size = 32
test_loader = DataLoader(
            test_dataset, # 验证数据.
            shuffle=False, # 打乱顺序
            batch_size = batch_size
        )

In [ ]:

mymodel.eval()
f=open('prediction.csv','w',encoding='utf_8')
f.write('{},{}\n'.format('ID','label'))
predictions=[]
r_acc = 0
with torch.no_grad():
    for i,batch in enumerate(test_loader):
        inputs_id, att_mask,rel_batch =batch
        inputs_id, att_mask = inputs_id.to(device), att_mask.to(device)
        pre=mymodel(inputs_id, att_mask)
        rel_tar = rel_batch.to(device)
        # rel_tar = rel_tar.unsqueeze(-1)
        rel=torch.argmax(pre,1)
        rel_acc = torch.eq(rel, rel_tar).sum().float().item()
        r_acc += rel_acc
        # pre=pre.squeeze(-1)
        # predictions.extend(pre.ge(0.5).int().cpu().numpy().tolist())
print('test_rel_acc={}'.format(r_acc/(len(test_dataset))))


# for i, pre in enumerate(predictions):
#     f.write('{},{}\n'.format(i, pre))

test_rel_acc=0.8997289972899729
